In [ ]:
pip install scikit-plot

In [ ]:
pip install neptune-client

     |████████████████████████████████| 245kB 15.2MB/s 
     |████████████████████████████████| 829kB 25.2MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 174kB 26.3MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 133kB 61.8MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.9.19-py2.py3-none-any.whl size=416637 sha256=0570a2c87d8835a4464a49db375b6e6a0e3301ba9c092e7dbb84435ac0a65b3a
  Stored in directory: /root/.cache/pip/wheels/01/b2/c3/fe12aedec3d73a79ec44d601b9c7fb9ed488ba77738b5fdbea
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=0efad845e98634fdb5347ccbc7aed0ff2afabe0a5572a2a34399e0001e840af7
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp37-none-a

In [ ]:
pip install psutil

In [ ]:
# Import Library yang Diperlukan

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.utils.vis_utils import plot_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
from tensorflow import keras
import neptune.new as neptune
from scikitplot.metrics import plot_confusion_matrix, plot_roc
from tensorflow.keras.callbacks import Callback
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import time

In [ ]:
# Fungsi untuk import file dataset

def load_file(filename):
	dataframe = read_csv(filename, header=None)
	return dataframe.values

In [ ]:
# Fungsi untuk menggabungkan dataset

def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [ ]:
# Fungsi untuk import dataset

def load_dataset_group():
  filenames = list()
  filenames += ['data Ir.csv']
  filenames += ['data Is.csv']
  filenames += ['data It.csv']
  filenames += ['data In.csv']
  filenames += ['data Vr.csv']
  filenames += ['data Vs.csv']
  filenames += ['data Vt.csv']
  # print(filenames)
  # load input data
  X = load_group(filenames)
  # load class output
  y = load_file('label.csv')
  return X, y

In [ ]:
# Fungsi mengambil dataset dan melakukan pembagian dataset

def load_dataset():
  # load all train
  X, y = load_dataset_group()
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.1, random_state=0)
  trainX, valX, trainy, valy = train_test_split(trainX, trainy, test_size=0.25, random_state=0)
  
  print(trainX.shape, trainy.shape, testX.shape, testy.shape, valX.shape, valy.shape)
  #print(testy)
  return trainX, trainy, testX, testy, valX, valy

In [ ]:
# Fungsi untuk pelatihan dataset

def evaluate_model(trainX, trainy, testX, testy, valX, valy):
  
  validation_data = valX, valy
  test_data = testX, testy

  verbose, epochs, batch_size = 1, 500, 0
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
  model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=([keras.metrics.Accuracy(name='accuracy', dtype=None),
                                                                            keras.metrics.Precision(name='precision'),
                                                                            keras.metrics.Recall(name='recall'),
                                                                            keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None)
                                                                            ]))
  
  neptune_logger_val = NeptuneLoggerCallback(model=model, validation_data=validation_data, state="validate")
  neptune_logger_test = NeptuneLoggerCallback(model=model, validation_data=test_data, state="testing")
  
  train_time = time.time()
  # fit network
  history = model.fit(trainX, trainy, validation_data=validation_data, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=neptune_logger_val)
  train_t = "{:.2f}".format(time.time()- train_time)
  run['time/Train Time'].log(train_t)
  print(f'Train Time : {train_t} s')

  evaluate_time = time.time()
  
  # evaluate model
  accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1, callbacks=neptune_logger_test, return_dict=True )
  eval_t = "{:.2f}".format(time.time()- evaluate_time)
  run['time/Evaluation Time'].log(eval_t)
  print(f'Evaluate Time : {eval_t} s')

  print(model.summary())
  plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
  model.save("drive/MyDrive/Model/model_112")


  return accuracy

In [ ]:
# Menghubungkan colab dan Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Fungsi logging

class NeptuneLoggerCallback(Callback):
    # mengambil data yang diperlukan
    def __init__(self, model, validation_data, state):
        super().__init__()
        self.model = model
        self.validation_data = validation_data
        self.state = state

    # berjalan setiap akhir dari epoch
    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
          run[f'report_{self.state}/{log_name}'].log(log_value)
           
        y_pred = np.asarray(self.model.predict(self.validation_data[0]))
        y_true = self.validation_data[1]

        y_true = np.argmax(y_true, axis=1)
        y_pred_class = np.argmax(y_pred, axis=1)
        
        # membuat confusion matrix
        target_names = ['Hubung Singkat', 'Non-System Fault', 'Petir']
        fig, ax = plt.subplots(figsize=(16, 12))
        plot_confusion_matrix(y_true, y_pred_class, ax=ax)
        run[f'report_{self.state}/confusion matrix Validation'].log(fig)

        # membuat log  Accuracy, Balance Accuracy, recal, precission, F1
        average = ['macro', 'micro', 'weighted']
        global bas, asc, prc, rcl, f1s, bac

        bas = balanced_accuracy_score(y_true, y_pred_class)
        asc = accuracy_score(y_true, y_pred_class)

        run[f'report_{self.state}/balanced_accuracy_score'].log(bas)
        run[f'report_{self.state}/accuracy_score'].log(asc)

        for av in average:
          prc = precision_score(y_true, y_pred_class, average=av, zero_division=1)
          rcl = recall_score(y_true, y_pred_class, average=av, zero_division=1)
          f1s = f1_score(y_true, y_pred_class, average=av, zero_division=1)

          run[f'report_{self.state}/precision_score_{av}'].log(prc)
          run[f'report_{self.state}/recall_score_{av}'].log(rcl)
          run[f'report_{self.state}/f1_score_{av}'].log(f1s)
          

        bac = classification_report(y_true, y_pred_class, target_names=target_names)
        run[f'report_{self.state}/classification_report'].log(bac)
        print("")
        print(f"================== validate Report epoch {epoch} ==================")
        print(bac)
        print("=============================================================")
        print("")

    # berjalan setiap akhir testing
    def on_test_end(self, logs={}):
      if self.state == "testing":
        for log_name, log_value in logs.items():
          run[f'report_{self.state}/{log_name}'].log(log_value)

        y_pred = np.asarray(self.model.predict(self.validation_data[0]))
        y_true = self.validation_data[1]

        y_true = np.argmax(y_true, axis=1)
        y_pred_class = np.argmax(y_pred, axis=1)

        target_names = ['Hubung Singkat', 'Non-System Fault', 'Petir']
        fig, ax = plt.subplots(figsize=(16, 12))
        plot_confusion_matrix(y_true, y_pred_class, ax=ax)
        run[f'report_{self.state}/confusion matrix testing'].log(fig)

        global bac
        
        bac = classification_report(y_true, y_pred_class, target_names=target_names)
        run[f'report_{self.state}/classification_report'].log(bac)
        
        print("")
        print("==================== Testing Report =====================")
        print(bac)
        print("=========================================================")
        print("")

In [ ]:
# Fungsi untuk menjalankan percobaan

def run_experiment():
  # load data
  trainX, trainy, testX, testy, valX, valy = load_dataset()
  score = evaluate_model(trainX, trainy, testX, testy, valX, valy)
  print(score)

In [ ]:
# Fungsi logging dan Menjalankan Percobaan

run = neptune.init(project='muflihnurfaizi/CNN', 
                   api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiMWRhYTU4MS00ZDc5LTQwOTgtYjgxOS04M2UzMzEyZjJkY2UifQ==')	

PARAMS = {'epoch': 500,
          'filter': 256,
          'kernel': 3,
          'pool_size': 2,
          'Dropuot': 0.5,
          'Dense':128,
          'Activation Function 1D': "relu",
          'Activation Function Dense': "relu"
          }

run['my_params'] = PARAMS

run_experiment()
run.stop()